In [16]:
%matplotlib inline  
import matplotlib.pyplot as plt
from pamtra2.libs.singleScattering import scattering
from pamtra2.libs.singleScattering import scattering_utilities as scautils
from pamtra2.libs.singleScattering import mie, rayleigh, tmatrix, ssrg
from pamtra2.libs import refractiveIndex
import numpy as np
np.set_printoptions(precision=3)
rad2deg = 180.0/np.pi

# Angles
t0 = np.pi*0.4 # theta must be within 0 pi
t1 = np.pi*0.6
#t1 = np.pi-t0
p0 = 0.2*np.pi
p1 = 0.4*np.pi
#p1 = np.pi+p0
print(p1, p1 // (2.*np.pi))
#p1 = p1 - 2.*np.pi*(p1//(2.*np.pi))
print(t0, t1, p0, p1)

1.2566370614359172 0.0
1.2566370614359172 1.8849555921538759 0.6283185307179586 1.2566370614359172


In [17]:
# One centimeter particle, S-band 2.8GHz, pure ice sphere 
f = 2.8e9
d = 0.001
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')

cost=1.0
R = rayleigh.RayleighScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
M = mie.MieScatt(d*cost,frequency=f/cost,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
T = tmatrix.TmatrixScatt(d*cost,frequency=f/cost,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
S = ssrg.SsrgScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1, volume=np.pi*d**3/6.)

from pytmatrix import tmatrix as tm
from pytmatrix import scatter, radar
scatt = tm.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(R.Cext, R.Csca, R.Cabs, R.Cbck)
print(M.Cext, M.Csca, M.Cabs, M.Cbck)
print(S.Cext, S.Csca, S.Cabs, S.Cbck)
print(T.Cext, T.Csca, T.Cabs, T.Cbck)


print(" ")
print("RAY")
print(" ")
print(R.S)
print(" ")
#print(R.S2/(R.S1*np.cos(p1-p0)))
print(" ")
print("MIE")
print(M.S)
print(" ")
#print(M.S2/(M.S1*np.cos(p1-p0)))
print(" ")
print("SSRG")
print(" ")
print(S.S)
print(" ")
print("TMM")
print(T.S)
print(" ")
#print(T.S[1,1]/(T.S[0,0]*np.cos(p1-p0)))
print(" ")
print(scatt.get_S())
print(" ")
print(T.S)
print(" ")
print(" ")


Cext                    Csca                    Cabs                   Cbck            [m**2]
3.42313882614e-12 2.74872213037e-13 3.1482666131e-12 4.12308319556e-13
3.42552492696e-12 2.74936863749e-13 3.15058806321e-12 4.12217233995e-13
3.42307506305e-12 2.748084499522858e-13 3.1482666131e-12 4.12117043789e-13
3.42552500089e-12 2.7493684988201866e-13 3.15058815101e-12 4.12217233682e-13
 
RAY
 
[[ -4.037e-08 -3.277e-12j  -3.290e-08 -2.670e-12j]
 [  1.480e-07 +1.201e-11j   1.465e-07 +1.189e-11j]]
 
 
MIE
[[ -4.038e-08 -3.564e-12j  -3.291e-08 -2.905e-12j]
 [  1.480e-07 +1.307e-11j   1.466e-07 +1.294e-11j]]
 
 
SSRG
 
[[ -4.037e-08 -3.277e-12j  -3.290e-08 -2.670e-12j]
 [  1.480e-07 +1.201e-11j   1.465e-07 +1.189e-11j]]
 
TMM
[[  1.499e-07 +1.323e-11j  -3.291e-08 -2.904e-12j]
 [ -3.291e-08 -2.904e-12j   1.466e-07 +1.294e-11j]]
 
 
[[  1.499e-07 +1.323e-11j  -3.291e-08 -2.904e-12j]
 [ -3.291e-08 -2.904e-12j   1.466e-07 +1.294e-11j]]
 
[[  1.499e-07 +1.323e-11j  -3.291e-08 -2.904e-12j]
 [ -3.

In [18]:
np.arcsin(np.sin(S.theta_inc)*np.sin(abs(S.phi_sca-S.phi_inc))/np.sin(S.scatt_angle)),S.rot_alpha,S.rot_beta,S.scatt_angle

(0.81047853428408223,
 0.81047853428408223,
 0.81047853428408223,
 0.88114109541898866)

In [12]:
RS = scautils.amplitude_matrix(R.S)
RZ = scautils.amplitude2mueller(RS)
SS = scautils.amplitude_matrix(S.S)
SZ = scautils.amplitude2mueller(SS)
TS = scautils.amplitude_matrix(T.S)
TZ = scautils.amplitude2mueller(TS)
MS = scautils.amplitude_matrix(M.S)
MZ = scautils.amplitude2mueller(MS)
print(RZ.matrix)
print("")
print(MZ.matrix)
print("")
print(SZ.matrix)
print("")
print("=================")
print("")
print(TZ.matrix)
print("")
print(scatt.get_Z())

[[  3.281e-14  -6.311e-30  -0.000e+00  -0.000e+00]
 [ -6.311e-30   3.281e-14  -0.000e+00  -0.000e+00]
 [ -0.000e+00  -0.000e+00   3.281e-14   0.000e+00]
 [ -0.000e+00  -0.000e+00   0.000e+00   3.281e-14]]

[[  3.283e-14  -4.293e-24  -0.000e+00  -0.000e+00]
 [ -4.293e-24   3.283e-14  -0.000e+00  -0.000e+00]
 [ -0.000e+00  -0.000e+00   3.283e-14   7.511e-32]
 [ -0.000e+00  -0.000e+00  -7.511e-32   3.283e-14]]

[[  3.281e-14  -6.311e-30  -0.000e+00  -0.000e+00]
 [ -6.311e-30   3.281e-14  -0.000e+00  -0.000e+00]
 [ -0.000e+00  -0.000e+00   3.281e-14   0.000e+00]
 [ -0.000e+00  -0.000e+00   0.000e+00   3.281e-14]]


[[  3.283e-14  -3.155e-30  -0.000e+00  -0.000e+00]
 [ -3.155e-30   3.283e-14  -0.000e+00  -0.000e+00]
 [ -0.000e+00  -0.000e+00   3.283e-14  -7.704e-34]
 [ -0.000e+00  -0.000e+00   7.704e-34   3.283e-14]]

[[  3.283e-14  -3.155e-30  -0.000e+00   0.000e+00]
 [ -3.155e-30   3.283e-14   0.000e+00   0.000e+00]
 [ -0.000e+00   0.000e+00   3.283e-14  -7.704e-34]
 [  0.000e+00   0.000e

In [5]:
from pamtra2.libs.singleScattering.scattering_utilities import rotation_matrix
Ra, Rb = rotation_matrix(S.rot_alpha, S.rot_beta)
print(Ra.T,"\n\n",Ra,"\n\n",Ra@Ra,"\n####\n",Rb.T,"\n\n",Rb,"\n\n",Rb.T@Rb)

[[-1. -0.]
 [-0. -1.]] 

 [[-1. -0.]
 [-0. -1.]] 

 [[ 1.  0.]
 [ 0.  1.]] 
####
 [[-1. -0.]
 [ 0. -1.]] 

 [[-1.  0.]
 [-0. -1.]] 

 [[ 1.  0.]
 [ 0.  1.]]


In [6]:
# One millimeter particle, W-band 94GHz, pure ice sphere 
f = 94e9
d = 0.001
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')
rayC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='TMM', theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC)
print(mieC)
print(TmaC)
from pytmatrix import tmatrix, scatter, radar
scatt = tmatrix.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)
print([scatter.ext_xsect(scatt),scatter.sca_xsect(scatt),scatter.ext_xsect(scatt)-scatter.sca_xsect(scatt),radar.radar_xsect(scatt)])

Cext                    Csca                    Cabs                   Cbck            [m**2]
(3.5159381620021679e-07, 3.4915146499913131e-07, 2.4423512010854983e-09, 5.2372719749869694e-07, array([[ 0. +3.829e-07j,  0. +0.000e+00j],
       [ 0. +0.000e+00j,  0. +3.829e-07j]]))
(3.8169026989341472e-07, 3.7719995847397647e-07, 4.4903114194382853e-09, 3.0198335883905455e-07, array([[ 0. +5.984e-05j,  0. +0.000e+00j],
       [ 0. +0.000e+00j,  0. +5.984e-05j]]))
(3.8169025547199794e-07, 3.7719994589127806e-07, 4.4903095807198777e-09, 3.019833724075803e-07, array([[ 0. +5.984e-05j,  0. +0.000e+00j],
       [ 0. +0.000e+00j,  0. +5.984e-05j]]))
[3.8169025547199794e-07, 3.7719994589127806e-07, 4.4903095807198777e-09, 8.89567423899513e-07]


In [7]:
# One millimeter particle, S-band 2.8GHz, pure ice sphere 
f=2.8e9
n = refractiveIndex.n(frequencies=f,temperatures=263.15,substance='ice')
rayC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='TMM')

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC)
print(mieC)
print(TmaC)

Cext                    Csca                    Cabs                   Cbck            [m**2]
(3.1485414853155008e-15, 2.7487221303700193e-19, 3.1482666131024637e-15, 4.1230831955550289e-19, array([[  1.811e-10 +1.470e-14j,   0.000e+00 +0.000e+00j],
       [  0.000e+00 +0.000e+00j,   1.811e-10 +1.470e-14j]]))
(3.1485646987497757e-15, 2.7487285968888451e-19, 3.1482898258900869e-15, 4.1230740899916799e-19, array([[  1.811e-10 +1.470e-14j,   0.000e+00 +0.000e+00j],
       [  0.000e+00 +0.000e+00j,   1.811e-10 +1.470e-14j]]))
(3.1485646291424952e-15, 2.7487285368539355e-19, 3.14828975628881e-15, 4.1230742079816574e-19, array([[  1.811e-10 +1.470e-14j,   0.000e+00 +0.000e+00j],
       [  0.000e+00 +0.000e+00j,   1.811e-10 +1.470e-14j]]))


In [8]:
scatt.get_S()
scatt.get_Z()

array([[  7.079e-08,   2.647e-23,  -0.000e+00,   0.000e+00],
       [  2.647e-23,   7.079e-08,   0.000e+00,   0.000e+00],
       [ -0.000e+00,   0.000e+00,   7.079e-08,  -6.617e-24],
       [  0.000e+00,   0.000e+00,   6.617e-24,   7.079e-08]])